# Cesar

In [3]:
import unicodedata

SPANISH_ALPHABET = "abcdefghijklmnñopqrstuvwxyz"

LETTER_FREQUENCIES = {
	'a': 11.525, 'b': 2.215, 'c': 4.019, 'd': 5.010, 'e': 12.181,
	'f': 0.692, 'g': 1.768, 'h': 0.703, 'i': 6.247, 'j': 0.493,
	'k': 0.011, 'l': 4.967, 'm': 3.157, 'n': 6.712, 'ñ': 0.311, 'o': 8.683,
	'p': 2.510, 'q': 0.877, 'r': 6.871, 's': 7.977, 't': 4.632,
	'u': 2.927, 'v': 1.138, 'w': 0.017, 'x': 0.215, 'y': 1.008,
	'z': 0.467, 'á': 0.502, 'é': 0.433, 'í': 0.725, 'ó': 0.827, 'ú': 0.168, 'ü': 0.012
}

def normalize_text(text):
	return ''.join(
		c for c in unicodedata.normalize('NFD', text.lower())
		if unicodedata.category(c) != 'Mn' and c in SPANISH_ALPHABET + " "
	)

def caesar_decrypt(ciphertext, key):
	decrypted_text = ''
	for char in ciphertext:
		if char in SPANISH_ALPHABET:
			new_index = (SPANISH_ALPHABET.index(char) - key) % len(SPANISH_ALPHABET)
			decrypted_text += SPANISH_ALPHABET[new_index]
		else:
			decrypted_text += char
	return decrypted_text

def calculate_metric(text):
	score = sum(LETTER_FREQUENCIES.get(char, 0) for char in text)
	return score

def brute_force_caesar(ciphertext):
	metrics = []

	for key in range(len(SPANISH_ALPHABET)):
		decrypted_text = caesar_decrypt(ciphertext, key)
		score = calculate_metric(decrypted_text)
		metrics.append((key, score, decrypted_text))

	metrics.sort(key=lambda x: x[1], reverse=True)
	
	return metrics[:5]
with open("ceasar.txt", "r", encoding="utf-8") as f:
	ciphertext = normalize_text(f.read())

best_caesar_keys = brute_force_caesar(ciphertext)

for key, score, text in best_caesar_keys:
	print(f"Key: {key}, Score: {score}")
	print(f"Decrypted Text: {text[:300]}")

Key: 23, Score: 1736.7990000000025
Decrypted Text: nnuestqolabeqintodigitalenconstanteevolucionlaagilidadcqiptogqaficacqiptoagilidadpaqaabqeviaqesunmecanismodedefensacqucialosbqindalacapacidaddemodificaqqapidamenteelusodealgoqitmosyclavescqiptogqaficosunaaccionnecesaqiapaqaanticipaqnosalasfutuqasamenazasdecibeqseguqidad
Key: 12, Score: 1203.4279999999999
Decrypted Text: xxfodebzvlmobsxezñsqselvoxnzxdelxeoogzvfnszxvllqsvsñlñnbsaezqblpsnlnbsaezlqsvsñlñalbllmbogslbodfxwonlxsdwzñoñopoxdlnbfnslvzdmbsxñlvlnlalnsñlññowzñspsnlbblasñlwoxeoovfdzñolvqzbsewzdjnvlgodnbsaezqblpsnzdfxllnnszxxonodlbslalbllxesnsalbxzdlvldpfefbldlwoxlkldñonsmobdoqfbsñlñ
Key: 19, Score: 1198.2410000000007
Decrypted Text: qqyiwxusoefiumqxshmkmxeoiqgsqwxeqxiizsoygmsqoeekmomhehgumtxskuejmgegumtxsekmomhehteueefuizmeuiwyqpigeqmwpshihijiqweguygmeoswfumqheoegetegmhehhipshmjmgeuuetmhepiqxiioywshieoksumxpswcgoeziwgumtxskuejmgswyqeeggmsqqigiweumeteueeqxmgmteuqsweoewjyxyuewepiqedewhigmfiuwikyumheh
Key: 0, Score: 1181.091000000002
D

# Afines

In [4]:
import math
import matplotlib.pyplot as plt
from collections import Counter

def gcd(a: int, b: int) -> int:
	while b:
		a, b = b, a % b
	return a

def mod_inverse(a: int, m: int) -> int:
	for i in range(1, m):
		if (a * i) % m == 1:
			return i
	return None

def affine_decrypt(text: str, a: int, b: int) -> str:
	a_inv = mod_inverse(a, 26)
	if a_inv is None:
		raise ValueError(f"No existe inverso para {a}.")
	
	decrypted_text = ""
	for char in text:
		if char.isalpha():
			y = ord(char.lower()) - ord('a')
			decrypted_char = (a_inv * (y - b)) % 26
			decrypted_text += chr(decrypted_char + ord('a'))
		else:
			decrypted_text += char
	return decrypted_text

def characters_frequency(text: str):
	frequency = Counter(text)
	total_chars = sum(frequency.values())
	return {char: count / total_chars for char, count in frequency.items() if char.isalpha()}

def compare_frequencies(actual_freq: dict, theoretical_freq: dict):
	all_chars = set(theoretical_freq.keys()).union(set(actual_freq.keys()))
	actual_freq = {char: actual_freq.get(char, 0) for char in all_chars}
	theoretical_freq = {char: theoretical_freq.get(char, 0) for char in all_chars}
	return math.sqrt(sum((actual_freq[char] - theoretical_freq[char]) ** 2 for char in all_chars))

def find_best_decryption(ciphertext: str):
	theoretical_freq = {
		'a': 0.1253, 'b': 0.0142, 'c': 0.0468, 'd': 0.0586, 'e': 0.1368, 'f': 0.0069,
		'g': 0.0101, 'h': 0.0070, 'i': 0.0625, 'j': 0.0044, 'k': 0.0002, 'l': 0.0497,
		'm': 0.0315, 'n': 0.0671, 'ñ': 0.0031, 'o': 0.0868, 'p': 0.0251, 'q': 0.0088, 'r': 0.0687,
		's': 0.0798, 't': 0.0463, 'u': 0.0393, 'v': 0.0090, 'w': 0.0001, 'x': 0.0022,
		'y': 0.0090, 'z': 0.0052
	}
	
	valid_a_values = [a for a in range(1, 17) if gcd(a, 26) == 1]
	valid_b_values = range(1, 17)
	
	best_text = None
	best_score = float("inf")
	best_a, best_b = None, None
	
	for a in valid_a_values:
		for b in valid_b_values:
			try:
				decrypted_text = affine_decrypt(ciphertext, a, b)
				actual_freq = characters_frequency(decrypted_text)
				score = compare_frequencies(actual_freq, theoretical_freq)
				
				if score < best_score:
					best_score = score
					best_text = decrypted_text
					best_a, best_b = a, b
			except ValueError:
				continue
	
	return best_text, best_a, best_b

with open("afines.txt", "r", encoding="utf-8") as file:
	contenido = file.read()
	print("Contenido cifrado:")
	print(contenido)
	print("Buscando la mejor desencriptación...")
	best_text, best_a, best_b = find_best_decryption(contenido)
	print(f"Mejor desencriptación encontrada con a={best_a}, b={best_b}:")
	print(best_text)

Contenido cifrado:
zigiuñpjdipoziyicbdoddiyxbñhjorbpbdodciñmidiiahxoixdipjcohosmicioxhtpiidmrdipixxjxxiqipjmzoditbpbdodyxmybopsmiñixubhbjopjcohoyozhicpiixpouiujxbodiubpicdicbchiuocfxjtoxbznjxuoybjzqopbjcoociuñxicocsmipjrxoxjzxiymñixoxcixoñbdouizhinmixjzpocsmidiujchxoxjzyxbñhjorbpbdodxiiuñpokozdjxoñbdouizhicmcypoqicyxbñhjrxonbyocfoprjxbhujcyjuñxjuihbdjcñjxjhxjczmiqjcfcirmxjcchibzybdizhicbxqiyjujmzypoxjigiuñpjdipobuñjxhozybodipoyxbñhjorbpbdodizzmichxotohoppoyjzhbzmoyjzhxopocouizokocdbzoubyocdiybtixcirmxbdod
Buscando la mejor desencriptación...
Mejor desencriptación encontrada con a=9, b=14:
hicisadltidahieiqntattiebnaflajndntatqiauitiikfbaibtidlqafamuiqiabfpdiitujtidibblbbigidluhatipndntatebuenadmuiaibsnfnladlqafaeahfiqdiibdasislbnatisndiqtiqnqfisaqzblpabnhxlbsaenlhgadnlqaaqisabiqaqmuidljbablhbieuaibabqibaantasihfixuiblhdaqmuitislqfbablhebnaflajndntatbiisadaoahtlbaantasihfiquqedagiqebnafljbaxneaqzadjlbnfslqelsablsifntlqalblfblqhuiglqzqijublqqfinhentihfiqnbgielsluhedablicisadltidansalbfah

# Vigenere

In [5]:
with open("vigenere.txt", "r", encoding="utf-8") as file:
	ciphertext = file.read().strip().lower()

SPANISH_ALPHABET = "abcdefghijklmnñopqrstuvwxyz"
MODULO = len(SPANISH_ALPHABET)

SPANISH_FREQ = {
	'a': 11.525, 'b': 2.215, 'c': 4.019, 'd': 5.010, 'e': 12.181, 'f': 0.692,
	'g': 1.768, 'h': 0.703, 'i': 6.247, 'j': 0.493, 'k': 0.011, 'l': 4.967,
	'm': 3.157, 'n': 6.712, 'ñ': 0.311, 'o': 8.683, 'p': 2.510, 'q': 0.877,
	'r': 6.871, 's': 7.977, 't': 4.632, 'u': 2.927, 'v': 1.138, 'w': 0.017,
	'x': 0.215, 'y': 1.008, 'z': 0.467
}

COMMON_BIGRAMS = ["de", "la", "el", "en", "es", "un", "se", "al", "lo", "le"]
COMMON_TRIGRAMS = ["que", "los", "del", "las", "por", "con", "una", "sus", "mas"]

def decrypt_vigenere(text, key):
	key = key.lower()
	if not all(k in SPANISH_ALPHABET for k in key):
		return None  
	
	key_indices = [SPANISH_ALPHABET.index(k) for k in key]
	key_length = len(key)
	decrypted_text = ""
	
	for i, c in enumerate(text):
		if c in SPANISH_ALPHABET:
			idx = (SPANISH_ALPHABET.index(c) - key_indices[i % key_length]) % MODULO
			decrypted_text += SPANISH_ALPHABET[idx]
		else:
			decrypted_text += c
	
	return decrypted_text

def frequency_score(text):
	text_freq = {}
	total_chars = 0
	
	for char in text:
		if char in SPANISH_FREQ:
			text_freq[char] = text_freq.get(char, 0) + 1
			total_chars += 1
	
	score = 0
	for char, count in text_freq.items():
		observed_freq = (count / total_chars) * 100
		expected_freq = SPANISH_FREQ.get(char, 0)
		score += abs(observed_freq - expected_freq)
	
	return score

def bigram_trigram_score(text):
	bigram_count = sum(text.count(bigram) for bigram in COMMON_BIGRAMS)
	trigram_count = sum(text.count(trigram) for trigram in COMMON_TRIGRAMS)
	return bigram_count + trigram_count

def generate_keys(prefix, length):
	keys = []
	
	def generate_recursive(current_key):
		if len(current_key) == length:
			keys.append(current_key)
			return
		for letter in SPANISH_ALPHABET:
			generate_recursive(current_key + letter)
	
	generate_recursive(prefix)
	return keys

def brute_force_attack(ciphertext, prefix, length, top_n=10):
	possible_keys = generate_keys(prefix, length)
	results = []
	
	for key in possible_keys:
		decrypted = decrypt_vigenere(ciphertext, key)
		if decrypted:
			freq_score = frequency_score(decrypted)
			struct_score = bigram_trigram_score(decrypted)
			results.append((key, decrypted, freq_score, struct_score))
	
	results.sort(key=lambda x: (-x[3], x[2]))
	return results[:top_n]

PREFIX = "pa"
KEY_LENGTH = 6
TOP_RESULTS = 10

top_keys = brute_force_attack(ciphertext, PREFIX, KEY_LENGTH, TOP_RESULTS)

print(f"Top {TOP_RESULTS} mejores claves encontradas:")
for i, (key, text, freq_score, struct_score) in enumerate(top_keys, start=1):
	print(f"{i}. Clave: {key}")
	print(f"Puntuación de estructuras comunes en español: {struct_score}")
	print(f"Métrica de similitud: {freq_score:.3f}")
	print(f"Texto descifrado:\n{text[:300]}...\n")

best_key, best_text, best_freq_score, best_struct_score = top_keys[0]
print(f"Mejor clave según estructura en español: {best_key}")
print(f"Puntuación de estructuras comunes en español: {best_struct_score}")
print(f"Métrica de similitud: {best_freq_score:.3f}")
print(f"Texto descifrado:\n{best_text[:300]}...\n")


Top 10 mejores claves encontradas:

 1. Clave: payaso
Puntuación de estructuras comunes en español: 186
Métrica de similitud: 17.718
Texto descifrado:
aunqueelerrorheartbleedsehasolucionadosiemprehayunanuevaamenazaenelhorizontehoylacuanticaesesaamenazaquepuedeatravesartodasnuestrasdefensasantesdequetodoesteperdidodebemosadoptarlacriptoagilidadparadefendernosdisponibleenlasprincipalessolucionesdecriptografiaposcuanticapqcdelaactualidadlaevidenciade...


 2. Clave: payjso
Puntuación de estructuras comunes en español: 157
Métrica de similitud: 20.626
Texto descifrado:
auniueelejrorherrtblevdsehakolucignadoszempreyayunaeuevaadenazavnelhojizontvhoylatuantitaesesramenaqaquepmedeatjavesajtodaseuestrrsdefeesasanlesdeqmetodovstepejdidodvbemosrdoptajlacrihtoagicidadprradefvnderngsdispgniblevnlaspjinciprlessocucionvsdecrzptogrrfiapokcuantzcapqcuelaaclualidrdlaevzdencirde...


 3. Clave: payhso
Puntuación de estructuras comunes en español: 153
Métrica de similitud: 20.047
Texto descifrado:
aunkue